We want to the quality of data augmentation on classification performance. For this purpose, consider the classes with small sample size (categories Disgust and Anticipation), identify from Kaggle or elsewhere labeled dataset of the same categories And add it to these two categories. Test the SVM classifier in 6) and discuss whether the overall performance can be increased or not. (Anh)

The dataset doesnt contain categories Disgust and Anticipation. So here I use love and sadness. Task 6 does not classify anything. Do it mean task 7 ??

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import random

In [151]:

def get_sample_data_for_categories(file_name,categories_to_extract):
    data = pd.read_csv('categories/{}.csv'.format(file_name))
    sample_size = 100 / len(categories_to_extract)
    
    df = pd.DataFrame()
    for category in categories_to_extract:
        index = data[data['Category'] == category].index[0]
        tweets = data['Concatenated_Tweets'][index].split(',')
        if len(tweets) >= int(sample_size):
            sampled_tweets = random.sample(tweets, int(sample_size))
        else:
            sampled_tweets = tweets
        
        for tweet in sampled_tweets:
            df = pd.concat([df, pd.DataFrame([{'category': category, 'tweet': tweet}])])
    return df

def get_sample_data_for_categories_addition(categories_to_extract):
    data = pd.read_csv('data/{}.csv'.format('text_emotion'))
    sample_size = 100
    sampled_data = data[data['sentiment'].isin(categories_to_extract)].sample(n=sample_size)
    selected_columns = sampled_data[['sentiment', 'content']]
    selected_columns = selected_columns.rename(columns={'sentiment': 'category', 'content': 'tweet'})
    return selected_columns
        
categories_to_extract = ['sadness', 'love']
sample_data = get_sample_data_for_categories('complete',categories_to_extract)
sample_data_addition = get_sample_data_for_categories_addition(categories_to_extract)

data = pd.concat([sample_data, sample_data_addition], ignore_index=True)
shuffled_df = data.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_df

,category,tweet
0,love,'ive lived my life trying so hard to be accep...
1,sadness,'im feeling pretty homesick this week but i s...
2,sadness,'i could clearly feel my adomen muscles contr...
3,sadness,@misssammibaby why are you sad?
4,sadness,R.i.p. to my lil sis's cousins babyfather Jimm...
...,...,...
195,sadness,i dont wanna miss the laker game tonight! i'm ...
196,sadness,'i feel like that enables her rotten ass even...
197,love,'i really am a hard worker and feel quite loy...
198,sadness,Approaching the weekend quite quickly and so m...


In [158]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

In [170]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
x = tfidf_vectorizer.fit_transform(data['tweet'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['category'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(x_train, y_train)

y_pred = svm_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)
f1 = f1_score(y_test, y_pred, average='weighted')


print("Accuracy:", accuracy)

df_report = pd.DataFrame(report).transpose()
df_report.to_csv('report/classification_report.csv', index=True)
print(df_report)


Accuracy: 0.675
              precision    recall  f1-score  support
love           0.857143  0.333333  0.480000   18.000
sadness        0.636364  0.954545  0.763636   22.000
accuracy       0.675000  0.675000  0.675000    0.675
macro avg      0.746753  0.643939  0.621818   40.000
weighted avg   0.735714  0.675000  0.636000   40.000
